# Matrix multiplication

Matrix multiplication is one of the most well-known linear algebra algorithms, and frequently used to demonstrate the high-performance computing capabilities of GPUs. As such, an example using matrix multiplication could not be left out. A naive CUDA kernel for a square matrix multiplication is:

In [ ]:
# %load matmul_naive.cu
#define WIDTH 4096

__global__ void matmul_kernel(float *C, float *A, float *B) {
    int x = blockIdx.x * block_size_x + threadIdx.x;
    int y = blockIdx.y * block_size_y + threadIdx.y;
    float sum = 0.0;

    for (int k=0; k<WIDTH; k++) {
        sum += A[y*WIDTH+k] * B[k*WIDTH+x];
    }

    C[y*WIDTH+x] = sum;
}


This kernel simply creates a single thread per output element. Each thread computes the index of the element it is responsible for, and iterates over the corresponding row in A, and corresponding column in B.

# Setup tuning parameters

Now we will explain how to use the kernel_tuner to tune all the parameters of this highly-flexible implementation. We'll first show the Python script and then explain it step-by-step.

In [2]:
#!/usr/bin/env python
import numpy
import kernel_tuner
from collections import OrderedDict

problem_size = (4096, 4096)
size = numpy.prod(problem_size)

A = numpy.random.randn(*problem_size).astype(numpy.float32)
B = numpy.random.randn(*problem_size).astype(numpy.float32)
C = numpy.zeros_like(A)

args = [C, A, B]

tune_params = OrderedDict()

tune_params["block_size_x"] = [2**i for i in range(3,7)]
tune_params["block_size_y"] = [2**i for i in range(6)]

answer = [numpy.dot(A,B), None, None]

As usual we first setup the kernel arguments and problem size so that the kernel_tuner knows how to call the kernel. Then for the block_size_x we choose a range of thread block sizes that seem reasonable, in this case [16, 32, 64]. You typically want the total number of threads within a thread block to be a multiple of 32 (warpsize) or even 64 (number of register banks on some cards). And because the tiling factors will increase the amount of work per thread block (see below), as well as the amount of shared memory used we start a tad conservatively here. For block_size_y, and the tiling factors in both directions, we just pick a range of powers of two.

In [3]:
results = kernel_tuner.tune_kernel("matmul_kernel", "matmul_naive.cu",
                                   problem_size, args, tune_params, answer=answer, atol=1e-3)

Using: GeForce GTX TITAN X
block_size_x=8, block_size_y=1, time=3345.30566406
block_size_x=8, block_size_y=2, time=1677.3567627
block_size_x=8, block_size_y=4, time=888.376782227
block_size_x=8, block_size_y=8, time=857.269091797
block_size_x=8, block_size_y=16, time=782.440197754
block_size_x=8, block_size_y=32, time=598.204455566
block_size_x=16, block_size_y=1, time=1630.11213379
block_size_x=16, block_size_y=2, time=821.787939453
block_size_x=16, block_size_y=4, time=783.853503418
block_size_x=16, block_size_y=8, time=710.253039551
block_size_x=16, block_size_y=16, time=582.648266602
block_size_x=16, block_size_y=32, time=497.817297363
block_size_x=32, block_size_y=1, time=983.634216309
block_size_x=32, block_size_y=2, time=845.382946777
block_size_x=32, block_size_y=4, time=814.851721191
block_size_x=32, block_size_y=8, time=620.501367188
block_size_x=32, block_size_y=16, time=548.021582031
block_size_x=32, block_size_y=32, time=515.425341797
block_size_x=64, block_size_y=1, time=

There aren't many parameters to tune yet, and more importantly, tuning will not be very effective because this kernel will be limited by bandwidth rather than compute. 

The utilisation of the GPU is very low, even for the optimal combination of block_size_x and block_size_y:

![](Matmul-naive-utilisation.png)

There is however, a lot of opportunity for data reuse, which is realized by making the threads in a thread block collaborate.

# Increase data reuse

This can be solved by using a technique called loop-blocking or loop-tiling. We define two square data structures in shared memory, which will be used for storing square parts of matrix A and B. The threads in a thread block will collaboratively fill these two variables, and then proceed to perform all the computations that need this data, before moving to the next blocked iteration.

In [ ]:
# %load matmul_data_reuse.cu
#define WIDTH 4096

__global__ void matmul_kernel(float *C, float *A, float *B) {

    __shared__ float sA[block_size_y][block_size_x];
    __shared__ float sB[block_size_y][block_size_x];

    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int x = blockIdx.x * block_size_x + tx;
    int y = blockIdx.y * block_size_y + ty;

    float sum = 0.0;
    int k,kb;

    for (k=0; k<WIDTH; k+=block_size_x) {
        __synchthreads();
        sA[ty][tx] = A[y*WIDTH+k+tx];
        sB[ty][tx] = B[(k+ty)*WIDTH+x];
        __synchthreads();

        for (kb=0; kb<block_size_x; kb++) {
            sum += sA[ty][kb] * sB[kb][tx];
        }

    }

    C[y*WIDTH+x] = sum;
}



In [4]:
restrict = ["block_size_x==block_size_y"] 

results = kernel_tuner.tune_kernel("matmul_kernel", "matmul_data_reuse.cu",
                                   problem_size, args, tune_params, restrictions=restrict, answer=answer, atol=1e-3)  

Using: GeForce GTX TITAN X
block_size_x=8, block_size_y=8, time=247.659780884
block_size_x=16, block_size_y=16, time=181.600396729
block_size_x=32, block_size_y=32, time=178.936386108
best performing configuration: block_size_x=32, block_size_y=32, time=178.936386108


This kernel drastically reduces memory bandwidth consumption. It is about three times faster now, which comes from much better memory use:

![](Matmul-utilisation-with-data-reuse.png)

The compute intensity has dropped slightly, because of the syncthread operations.

However, there still is not too much that can be tuned in this kernel. In fact, because the thread block size needs to be a square, there only a handful of configurations we can try. Fortunately, we can add several more optimizations to the code that also open the parameter space for tuning.

# Increase work per thread

We will use two different forms of 1xN tiling in this example:

First of all, in the x-direction we will use tiling in a way that is similar to the convolution example. The area of output data that is processed by a single thread block is increased by a factor of N, and as such shared memory usage also increases by a factor N. This means that the number of thread blocks needed to execute the kernel for this problem size is reduced by a factor of N, where N is the tiling factor. While this may reduce occupancy due to increased shared memory and register usage, this optimization drastically reduces the number of redundant instructions that were previously distributed across multiple thread blocks.

Secondly, in the y-direction we will use a different form of 1xN tiling, where we tile within the thread block. This too means that threads will compute multiple elements, but in this case, not the total number of thread blocks is reduced, but instead the number of threads per block goes down.

Note that these two different forms of tiling could have combined in different or even multiple ways to increase the tuning parameter space even further. However, for the purposes of this example, the resulting kernel is already complex enough:

In [ ]:
# %load matmul_increase_work_per_thread.cu
#define WIDTH 4096

__global__ void matmul_kernel(float *C, float *A, float *B) {

    __shared__ float sA[block_size_y*tile_size_y][block_size_x];
    __shared__ float sB[block_size_y*tile_size_y][block_size_x * tile_size_x];

    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int x = blockIdx.x * block_size_x * tile_size_x + threadIdx.x;
    int y = blockIdx.y * block_size_y * tile_size_y + threadIdx.y;
    int k, kb;

    float sum[tile_size_y][tile_size_x];

    for (k = 0; k < WIDTH; k += block_size_x) {

        __syncthreads ();
        #pragma unroll
        for (int i = 0; i < tile_size_y; i++) {
            sA[ty + block_size_y * i][tx] = A[y * WIDTH + block_size_y * i * WIDTH + k + tx];

            #pragma unroll
            for (int j = 0; j < tile_size_x; j++) {
                sB[ty + block_size_y * i][tx + j * block_size_x] = B[(k + ty + block_size_y * i) * WIDTH + x + j * block_size_x];
            }
        }
        __syncthreads ();

        //compute
        #pragma unroll
        for (kb = 0; kb < block_size_x; kb++) {

            #pragma unroll
            for (int i = 0; i < tile_size_y; i++) {
                #pragma unroll
                for (int j = 0; j < tile_size_x; j++) {
                        sum[i][j] += sA[ty + block_size_y * i][kb] * sB[kb][tx + j * block_size_x];
                    }
            }

        }

    }

    //store result
    #pragma unroll
    for (int i = 0; i < tile_size_y; i++) {
        #pragma unroll
        for (int j = 0; j < tile_size_x; j++) {
            C[y * WIDTH + x + block_size_y * i * WIDTH + j * block_size_x] = sum[i][j];
        }
    }

}


Remember that the area operated on by the thread block should be a square. In this kernel however, we allow block_size_x and block_size_y to vary independently, while tile_size_y increases the amount of work per thread in the y-direction within the thread block. This yields a discontinuous search space in which only part of the configurations are actually valid. Therefore we use the restrictions optional argument of tune_kernel.

restrictions expects a list of strings that contain a boolean expression that may use the tuning parameters as variables. Any occurrences of tuning parameter names will be replaced with the specific value of this parameter when the kernel configuration is evaluated. All expressions in the list passed as restrictions need to evaluate to True for the configuration to be considered valid and therefore part of the parameter space.

In [6]:
tune_params["tile_size_x"] = [2**i for i in range(4)]
tune_params["tile_size_y"] = [2**i for i in range(4)]

grid_div_x = ["block_size_x", "tile_size_x"]
grid_div_y = ["block_size_y", "tile_size_y"]

restrict = ["block_size_x==block_size_y*tile_size_y"]

results = kernel_tuner.tune_kernel("matmul_kernel", "matmul_increase_work_per_thread.cu",
    problem_size, args, tune_params,
    grid_div_y=grid_div_y, grid_div_x=grid_div_x,
    restrictions=restrict, verbose=True)

Using: GeForce GTX TITAN X
skipping config 8_1_1_1 reason: config fails restriction
skipping config 8_1_1_2 reason: config fails restriction
skipping config 8_1_1_4 reason: config fails restriction
block_size_x=8, block_size_y=1, tile_size_x=1, tile_size_y=8, time=489.548773193
skipping config 8_1_2_1 reason: config fails restriction
skipping config 8_1_2_2 reason: config fails restriction
skipping config 8_1_2_4 reason: config fails restriction
block_size_x=8, block_size_y=1, tile_size_x=2, tile_size_y=8, time=302.98302002
skipping config 8_1_4_1 reason: config fails restriction
skipping config 8_1_4_2 reason: config fails restriction
skipping config 8_1_4_4 reason: config fails restriction
block_size_x=8, block_size_y=1, tile_size_x=4, tile_size_y=8, time=233.32043457
skipping config 8_1_8_1 reason: config fails restriction
skipping config 8_1_8_2 reason: config fails restriction
skipping config 8_1_8_4 reason: config fails restriction
block_size_x=8, block_size_y=1, tile_size_x=8, t

In [13]:
params = {"block_size_x": 32, "block_size_y": 4, "tile_size_x": 4, "tile_size_y":8} 

In [14]:
results = kernel_tuner.run_kernel("matmul_kernel", "matmul_increase_work_per_thread.cu", problem_size, args, params)

Using: GeForce GTX TITAN X


LogicError: cuMemcpyDtoH failed: an illegal memory access was encountered

In [9]:
results = kernel_tuner.tune_kernel("matmul_kernel", "matmul_increase_work_per_thread.cu",
    problem_size, args, tune_params,
    grid_div_y=grid_div_y, grid_div_x=grid_div_x,
    restrictions=restrict, verbose=True)

Using: GeForce GTX TITAN X
skipping config 8_1_1_1 reason: config fails restriction
skipping config 8_1_1_2 reason: config fails restriction
skipping config 8_1_1_4 reason: config fails restriction
block_size_x=8, block_size_y=1, tile_size_x=1, tile_size_y=8, time=489.201403809
skipping config 8_1_2_1 reason: config fails restriction
skipping config 8_1_2_2 reason: config fails restriction
skipping config 8_1_2_4 reason: config fails restriction
block_size_x=8, block_size_y=1, tile_size_x=2, tile_size_y=8, time=300.227099609
skipping config 8_1_4_1 reason: config fails restriction
skipping config 8_1_4_2 reason: config fails restriction
skipping config 8_1_4_4 reason: config fails restriction
block_size_x=8, block_size_y=1, tile_size_x=4, tile_size_y=8, time=233.216320801
skipping config 8_1_8_1 reason: config fails restriction
skipping config 8_1_8_2 reason: config fails restriction
skipping config 8_1_8_4 reason: config fails restriction
block_size_x=8, block_size_y=1, tile_size_x=8,